In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from math import sqrt

In [2]:
data=pd.read_csv("./merged_final_data.csv",dtype=str)
data['hospital system or, if independent, ipps/cah'].nunique()
print(data.shape)

(3329, 87)


In [3]:
data=data.drop(columns=['hospital name','city','hospital system or, if independent, ipps/cah',
                       'state','street address'])



In [4]:
#print number of Nan per coulmns
nan_col=data.columns[data.isnull().any()]
for n in nan_col:
    print(n,data[n].isnull().sum())
    
#drop columns have more than 70% Nan
for n in nan_col:
    if data[n].isnull().sum()/len(data)>0.7:
        data.drop(n,1,inplace=True)
data.shape

hospital compare 5-star rating (october 2018, na=not available) 476
number of outpatient services 37
total private allowed amount for outpatient services ($ millions) 37
simulated medicare allowed amount for outpatient services ($ millions) 37
relative price for outpatient services 37
standardized price per outpatient service 37
number of inpatient stays 1664
total private allowed amount for inpatient services ($ millions) 1664
simulated medicare allowed amount for inpatient services ($ millions) 1664
relative price for inpatient services 1664
standardized price per inpatient stay 1664
total private allowed amount for inpatient and outpatient services ($ millions) 1701
simulated medicare allowed amount for inpatient and outpatient services ($ millions) 1701
relative price for professional inpatient and outpatient services 1743
relative price for inpatient facility services 1709
relative price for outpatient facility services 251
relative price for inpatient and outpatient services 1701

(3329, 52)

In [5]:
# rearrange column (put reponse colkumn to the end)
data = data[[c for c in data if c not in ['relative price for inpatient and outpatient services']] 
       + ['relative price for inpatient and outpatient services']]
# remove % at the end of these 2 columns
data['relative price for inpatient facility services'] = data['relative price for inpatient facility services'].str.rstrip('%').astype('float')
data['relative price for outpatient facility services'] = data['relative price for outpatient facility services'].str.rstrip('%').astype('float')

# convert object data type column to dummy variables
labelencoder = LabelEncoder()
cat_features=[x for x in data.columns if data[x].dtype=="object"]
for col in cat_features:
    if col in data.columns:
        i = data.columns.get_loc(col)
        data.iloc[:,i] = data.apply(lambda i:labelencoder.fit_transform(i.astype(str)), axis=0, result_type='expand')

# convert all to numeric

data = data.apply(pd.to_numeric, errors='ignore')
data

,medicare provider number,zip code,is hospital a critical access hospital (y/n)?,"hospital compare 5-star rating (october 2018, na=not available)",number of outpatient services,total private allowed amount for outpatient services ($ millions),simulated medicare allowed amount for outpatient services ($ millions),relative price for outpatient services,standardized price per outpatient service,number of inpatient stays,...,Average Medicare Payment Amount.1,Average Medicare Payment Amount.2,Outlier Comprehensive APC Services,Outlier Comprehensive APC Services.1,Outlier Comprehensive APC Services.2,Average Medicare Outlier Amount,Average Medicare Outlier Amount.1,Average Medicare Outlier Amount.2,medicare provider number.1,relative price for inpatient and outpatient services
0,0,844,0,1,898,11,9,26,3120,504,...,2291,2288,263,552,697,448,992,975,1277,340
1,1,834,0,0,783,4,2,120,675,504,...,2291,2288,263,552,697,448,992,975,1277,340
2,2,826,0,2,84,9,4,98,280,330,...,2291,2288,263,552,697,448,992,975,1277,129
3,3,820,0,1,581,6,4,72,127,504,...,2291,2288,263,552,697,448,992,975,1277,340
4,4,835,0,1,303,3,2,17,3126,504,...,2291,2288,263,552,697,448,992,975,1277,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3324,3318,2198,0,5,1188,10,0,359,2214,504,...,134,196,263,237,231,448,382,348,1277,340
3325,3320,2378,0,5,960,10,0,415,2573,504,...,2291,2288,263,552,697,448,992,975,1277,340
3326,3322,2214,0,5,443,0,0,296,1761,504,...,49,199,263,552,0,448,992,0,1277,340
3327,3326,2259,0,5,663,40,10,443,2737,504,...,128,137,263,552,0,448,992,0,1277,340


In [6]:
# filter to only labeled data
idx = data['relative price for inpatient and outpatient services'].isnull()
dt = data[~idx].reset_index(drop=True)
# fill NaN with imputation
dt.fillna(method="ffill", inplace=True)
dt.fillna(method="bfill", inplace=True)
print(dt.shape)
dt

(3329, 52)


,medicare provider number,zip code,is hospital a critical access hospital (y/n)?,"hospital compare 5-star rating (october 2018, na=not available)",number of outpatient services,total private allowed amount for outpatient services ($ millions),simulated medicare allowed amount for outpatient services ($ millions),relative price for outpatient services,standardized price per outpatient service,number of inpatient stays,...,Average Medicare Payment Amount.1,Average Medicare Payment Amount.2,Outlier Comprehensive APC Services,Outlier Comprehensive APC Services.1,Outlier Comprehensive APC Services.2,Average Medicare Outlier Amount,Average Medicare Outlier Amount.1,Average Medicare Outlier Amount.2,medicare provider number.1,relative price for inpatient and outpatient services
0,0,844,0,1,898,11,9,26,3120,504,...,2291,2288,263,552,697,448,992,975,1277,340
1,1,834,0,0,783,4,2,120,675,504,...,2291,2288,263,552,697,448,992,975,1277,340
2,2,826,0,2,84,9,4,98,280,330,...,2291,2288,263,552,697,448,992,975,1277,129
3,3,820,0,1,581,6,4,72,127,504,...,2291,2288,263,552,697,448,992,975,1277,340
4,4,835,0,1,303,3,2,17,3126,504,...,2291,2288,263,552,697,448,992,975,1277,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3324,3318,2198,0,5,1188,10,0,359,2214,504,...,134,196,263,237,231,448,382,348,1277,340
3325,3320,2378,0,5,960,10,0,415,2573,504,...,2291,2288,263,552,697,448,992,975,1277,340
3326,3322,2214,0,5,443,0,0,296,1761,504,...,49,199,263,552,0,448,992,0,1277,340
3327,3326,2259,0,5,663,40,10,443,2737,504,...,128,137,263,552,0,448,992,0,1277,340


In [7]:
# split dataset to x and y
random_state = 100
x_data = dt.loc[:, dt.columns != "relative price for inpatient and outpatient services"]
y_data = dt.loc[:, "relative price for inpatient and outpatient services"]

In [23]:
#split dataset
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, shuffle=True, random_state=random_state)

regr = RandomForestRegressor(max_depth=None, random_state= random_state, n_estimators=100)
regr.fit(x_train, y_train)
y_pred = regr.predict(x_train)
rmse_random_forest_train = sqrt(mean_squared_error(y_train, y_pred))

print("RF MSE",rmse_random_forest_train) 

count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for ranrom forest: ',accuracy,"%")

RF MSE 6.454232799772088
accuracy for ranrom forest:  57.57211538461539 %


In [9]:
#elastic net
en = ElasticNet(random_state=12345678)
en.fit(x_train,y_train)
y_pred = en.predict(x_train)
print('MSE for Elastic Net: '+ str(mean_squared_error(y_train, y_pred)))


count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for elastic net: ',accuracy,"%")

MSE for Elastic Net: 1194.5449561913947
accuracy for elastic net:  1.6025641025641024 %


In [10]:
#Lasso
lasso_clf = LassoCV(cv=5, random_state=12345678).fit(x_train, y_train)
y_pred = lasso_clf.predict(x_train)
#print("Lasso accuracy",accuracy_score(y_test,y_pred))
print('MSE for Lasso: '+ str(mean_squared_error(y_train, y_pred)))
count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for Lasso: ',accuracy,"%")

MSE for Lasso: 1377.2885703384406
accuracy for Lasso:  1.4022435897435899 %


In [11]:
#ridge
ridge_clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(x_train, y_train)
y_pred = ridge_clf.predict(x_train)
#print("Ridge accuracy",accuracy_score(y_test,y_pred))
print('MSE for Ridge: '+ str(mean_squared_error(y_train, y_pred)))
count=0
for i,j in zip(y_pred,y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for ridge: ',accuracy,"%")

MSE for Ridge: 1193.441244368235
accuracy for ridge:  1.4823717948717947 %


In [12]:
#ridge
ridge_clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(x_train, y_train)
y_pred = ridge_clf.predict(x_train)
#print("Ridge accuracy",accuracy_score(y_test,y_pred))
print('MSE for Ridge: '+ str(mean_squared_error(y_train, y_pred)))
count=0
for i,j in zip(y_pred,y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for ridge: ',accuracy,"%")

MSE for Ridge: 1193.441244368235
accuracy for ridge:  1.4823717948717947 %


In [13]:
#==============================================

In [14]:
#below code were trying to improve perfomance, but FAILED

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=12345678).fit(x_train,y_train)
y_predict_train=rf_clf.predict(x_train)
y_predict_test=rf_clf.predict(x_test)
#print("train set accuracy ",accuracy_score(y_train,y_predict_train)*100)
print("random forest accuracy", accuracy_score(y_test,y_predict_test )*100,"%")
print('MSE for rf: '+ str(mean_squared_error(y_train, y_pred)))
#get most important features
np.argsort(rf_clf.feature_importances_)

random forest accuracy 51.7406962785114 %
MSE for rf: 1193.441244368235


array([ 2, 44, 47, 46, 49, 48, 45,  3, 35, 40, 43, 38, 42, 41, 36, 39, 37,
       26, 34, 27, 25, 28, 30, 24, 23, 33, 29, 32, 31,  6,  1,  5,  4,  0,
        8, 18, 17,  7, 50, 11, 13, 19,  9, 10, 12, 20, 21, 22, 16, 15, 14])

In [16]:
#get top 30 features
dt.columns[[28, 30, 24, 23, 33, 29, 32, 31,  6,  1,  5,  4,  0,
        8, 18, 17,  7, 50, 11, 13, 19,  9, 10, 12, 20, 21, 22, 16, 15, 14]]

Index(['Average Covered Charges.2', 'Average Total Payments.1',
       'Total Discharges.1', 'Total Discharges', 'Average Medicare Payments.1',
       'Average Total Payments', 'Average Medicare Payments',
       'Average Total Payments.2',
       'simulated medicare allowed amount for outpatient services ($ millions)',
       'zip code',
       'total private allowed amount for outpatient services ($ millions)',
       'number of outpatient services', 'medicare provider number',
       'standardized price per outpatient service',
       'relative price for outpatient facility services',
       'relative price for inpatient facility services',
       'relative price for outpatient services', 'medicare provider number.1',
       'simulated medicare allowed amount for inpatient services ($ millions)',
       'standardized price per inpatient stay',
       'total private allowed amount for facility inpatient and outpatient services ($ millions)',
       'number of inpatient stays',
      

In [17]:
dt2=dt[['Average Covered Charges.2', 'Average Total Payments.1',
       'Total Discharges.1', 'Total Discharges', 'Average Medicare Payments.1',
       'Average Total Payments', 'Average Medicare Payments',
       'Average Total Payments.2',
       'simulated medicare allowed amount for outpatient services ($ millions)',
       'zip code',
       'total private allowed amount for outpatient services ($ millions)',
       'number of outpatient services', 'medicare provider number',
       'standardized price per outpatient service',
       'relative price for outpatient facility services',
       'relative price for inpatient facility services',
       'relative price for outpatient services', 'medicare provider number.1',
       'simulated medicare allowed amount for inpatient services ($ millions)',
       'standardized price per inpatient stay',
       'total private allowed amount for facility inpatient and outpatient services ($ millions)',
       'number of inpatient stays',
       'total private allowed amount for inpatient services ($ millions)',
       'relative price for inpatient services',
       'simulated medicare allowed amount for facility inpatient and outpatient services ($ millions)',
       'total private allowed amount for professional inpatient and outpatient services ($ millions)',
       'simulated medicare allowed amount for professional inpatient and outpatient services ($ millions)',
       'relative price for professional inpatient and outpatient services',
       'simulated medicare allowed amount for inpatient and outpatient services ($ millions)',
       'total private allowed amount for inpatient and outpatient services ($ millions)','relative price for inpatient and outpatient services']]
dt2

,Average Covered Charges.2,Average Total Payments.1,Total Discharges.1,Total Discharges,Average Medicare Payments.1,Average Total Payments,Average Medicare Payments,Average Total Payments.2,simulated medicare allowed amount for outpatient services ($ millions),zip code,...,number of inpatient stays,total private allowed amount for inpatient services ($ millions),relative price for inpatient services,simulated medicare allowed amount for facility inpatient and outpatient services ($ millions),total private allowed amount for professional inpatient and outpatient services ($ millions),simulated medicare allowed amount for professional inpatient and outpatient services ($ millions),relative price for professional inpatient and outpatient services,simulated medicare allowed amount for inpatient and outpatient services ($ millions),total private allowed amount for inpatient and outpatient services ($ millions),relative price for inpatient and outpatient services
0,1760,13,2215,2245,2417,2703,2325,33,9,844,...,504,686,349,604,409,349,242,659,905,340
1,440,1850,1849,2028,1370,1915,1430,1872,2,834,...,504,686,349,604,409,349,242,659,905,340
2,1502,2731,2013,2122,2349,2645,2248,2306,4,826,...,330,48,149,9,8,6,37,15,44,129
3,1835,317,1328,1441,2632,98,2428,272,4,820,...,504,686,349,604,409,349,242,659,905,340
4,1816,1817,153,224,1411,1531,1051,1716,2,835,...,504,686,349,604,409,349,242,659,905,340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3324,2728,2733,2446,2456,2733,2736,2736,2728,0,2198,...,504,686,349,604,409,349,242,659,905,340
3325,2728,2733,2446,2456,2733,2736,2736,2728,0,2378,...,504,686,349,604,409,349,242,659,905,340
3326,539,2733,2446,2456,2733,2736,2736,1575,0,2214,...,504,686,349,604,409,349,242,659,905,340
3327,2728,2733,2446,2456,2733,2736,2736,2728,10,2259,...,504,686,349,604,409,349,242,659,905,340


In [18]:
#re-train models
random_state = 100
x_data = dt2.loc[:, dt2.columns != "relative price for inpatient and outpatient services"]
y_data = dt2.loc[:, "relative price for inpatient and outpatient services"]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, shuffle=True, random_state=random_state)

#Random forest
regr = RandomForestRegressor(max_depth=None, random_state= random_state, n_estimators=100)
regr.fit(x_train, y_train)
y_pred = regr.predict(x_train)
rmse_random_forest_train = sqrt(mean_squared_error(y_train, y_pred))

print("RF MSE",rmse_random_forest_train) 

count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for elastic net: ',accuracy,"%")

RF MSE 6.454232799772088
accuracy for elastic net:  57.57211538461539 %


In [19]:
#elastic net
en = ElasticNet(random_state=12345678)
en.fit(x_train,y_train)
y_pred = en.predict(x_train)
print('MSE for Elastic Net: '+ str(mean_squared_error(y_train, y_pred)))


count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for elastic net: ',accuracy,"%")

MSE for Elastic Net: 1208.3390870001156
accuracy for elastic net:  1.5625 %


In [20]:
#Lasso
lasso_clf = LassoCV(cv=5, random_state=12345678).fit(x_train, y_train)
y_pred = lasso_clf.predict(x_train)
#print("Lasso accuracy",accuracy_score(y_test,y_pred))
print('MSE for Lasso: '+ str(mean_squared_error(y_train, y_pred)))
count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for Lasso: ',accuracy,"%")

MSE for Lasso: 1398.0693631886957
accuracy for Lasso:  1.1217948717948718 %


In [21]:
#ridge
ridge_clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(x_train, y_train)
y_pred = ridge_clf.predict(x_train)
#print("Ridge accuracy",accuracy_score(y_test,y_pred))
print('MSE for Ridge: '+ str(mean_squared_error(y_train, y_pred)))
count=0
for i,j in zip(y_pred,y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for ridge: ',accuracy,"%")

MSE for Ridge: 1208.4497943783608
accuracy for ridge:  1.6025641025641024 %


In [22]:
#DT
cart_clf = DecisionTreeClassifier(max_depth=3, random_state=12345678).fit(x_train, y_train)
y_pred = cart_clf.predict(x_train)
print('MSE for Decision Tree: '+ str(mean_squared_error(y_train, y_pred)))
count=0
for i,j in zip(np.round(y_pred),y_train.to_numpy()):
    if i==j:
        count+=1
accuracy=count/len(y_train)*100
print('accuracy for Decision Tree: ',accuracy,"%")

MSE for Decision Tree: 1755.673076923077
accuracy for Decision Tree:  52.76442307692307 %
